In [2]:
import pandas as pd
import os
import random
import time

In [21]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_timestamp.csv')
df = pd.read_csv(csv_file_path) 
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,wday,retry,time,time_class
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,1,0.0,3.0,0
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,1,0.0,8.0,0
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,1,0.0,7.0,0
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,1,0.0,7.0,0
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,1,0.0,11.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,4,0.0,27.0,2
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,4,0.0,11.0,1
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,4,0.0,46.0,3
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,4,0.0,73.0,4


In [22]:
def testid_fe(df):
    ## test
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum','std'])
    correct_t.columns = ["test_ans_mean", 'test_ans_sum','test_ans_std']

    time_t = df.groupby(['testId'])['time'].agg(['mean', 'std'])
    time_t.columns = ["test_time_mean", 'test_time_std']

    ## 시험지 대분류(test_type)
    df['test_type'] = df['testId'].apply(lambda x:x[2])
    correct_type = df.groupby(['test_type'])['answerCode'].agg(['mean', 'sum','std'])
    correct_type.columns = ["t_type_ans_mean", 't_type_ans_sum','t_type_ans_std']

    time_type = df.groupby(['test_type'])['time'].agg(['mean', 'std'])
    time_type.columns = ["t_type_time_mean", 't_type_time_std']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, time_t, on=['testId'], how="left")
    df = pd.merge(df, correct_type, on=['test_type'], how="left")
    df = pd.merge(df, time_type, on=['test_type'], how="left")
    
    return df

In [23]:
df = testid_fe(df)

In [24]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,time_class,test_type,test_ans_mean,test_ans_sum,test_ans_std,test_time_mean,test_time_std,t_type_ans_mean,t_type_ans_sum,t_type_ans_std
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,0,6,0.947683,1268,0.222749,22.549327,32.154489,0.709232,187545,0.454118
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,...,0,6,0.947683,1268,0.222749,22.549327,32.154489,0.709232,187545,0.454118
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,...,0,6,0.947683,1268,0.222749,22.549327,32.154489,0.709232,187545,0.454118
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,...,0,6,0.947683,1268,0.222749,22.549327,32.154489,0.709232,187545,0.454118
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,...,1,6,0.947683,1268,0.222749,22.549327,32.154489,0.709232,187545,0.454118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,...,2,3,0.662590,921,0.472996,50.710791,58.611862,0.702238,192246,0.457275
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,...,1,4,0.655109,718,0.475550,46.906022,58.021546,0.684056,182864,0.464891
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,...,3,4,0.655109,718,0.475550,46.906022,58.021546,0.684056,182864,0.464891
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,...,4,4,0.655109,718,0.475550,46.906022,58.021546,0.684056,182864,0.464891
